In [268]:
'''import of the modules'''
from sklearn.model_selection import GridSearchCV
import polars as pl
import numpy as np
import os
from skimage import transform, color
from skimage.io import imread, imshow
import skimage.measure
import cv2
import torch
from torchvision.io import read_image
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import piq
import piqa

In [319]:
data_dir = 'D:/Практика/dataset'

In [320]:
data = {}
fid_metric = piqa.FID()
lab = -1
list_feats = []
list_labels = []
for category in os.listdir(data_dir):
  lab += 1
  for file_name in os.listdir(os.path.join(data_dir, category)):
    #print(file_name)
    img_path = os.path.join(data_dir, category, file_name)
    img_array = imread(img_path, as_gray=False)
    if len(img_array.shape) != 3 or img_array.shape[2] != 3:
      img_array =  cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    size = (250, 250)
    resized_img = cv2.resize(img_array, size)
    #bright_img = cv2.add(resized_img, 50)
    img_tensor = torch.tensor(resized_img).permute(2, 0, 1)[None, ...] / 255
    img_feats = fid_metric.features(img_tensor).reshape(-1)
    list_feats.append(img_feats)
    list_labels.append(lab)
data['features'] = list_feats
data['labels'] = list_labels


In [321]:
# Разделение данных на обучающий, валидационный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(data['features'], data['labels'], test_size=0.2, shuffle=True, stratify=data['labels'])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, shuffle=True, stratify=y_train)


попробовать через cuda и на линуксе

In [322]:
model = RandomForestClassifier()

In [323]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy', 'log_loss']
}

In [324]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [325]:
best_params = grid_search.best_params_
print(f'Лучшие параметры: {best_params}')

# Получение лучшей модели
best_model = grid_search.best_estimator_

Лучшие параметры: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 200}


In [326]:
# Оценка производительности модели на валидационном наборе
val_preds = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_preds)
print(f'Accuracy на валидационном наборе: {val_accuracy}')

Accuracy на валидационном наборе: 0.9009009009009009


In [327]:
# Оценка производительности модели на тестовом наборе
test_preds = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_preds)
print(f'Accuracy на тестовом наборе: {test_accuracy}')
print('Precision:', precision_score(y_test, test_preds, average='weighted'))
print('Recall:', recall_score(y_test, test_preds, average='weighted'))
print('F1:', f1_score(y_test, test_preds, average='weighted'))

Accuracy на тестовом наборе: 0.9372197309417041
Precision: 0.9381307094311578
Recall: 0.9372197309417041
F1: 0.9367611088463107


In [328]:
test_preds = best_model.predict(X_train)
test_accuracy = accuracy_score(y_train, test_preds)
print(f'Accuracy на обучающем наборе: {test_accuracy}')

Accuracy на обучающем наборе: 1.0


In [279]:
# saving the model 0.85 
import pickle

In [366]:
with open('final_model.p', 'wb') as f:
  pickle.dump(best_clf, f)
  f.close()

попробовать модель SVC

In [329]:
from sklearn import svm

In [354]:
X_train, X_test, y_train, y_test = train_test_split(data['features'], data['labels'], test_size=0.2, shuffle=True, stratify=data['labels'])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, shuffle=True, stratify=y_train)

In [355]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

In [356]:
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=3)
clf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=SVC(probability=True),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

In [361]:
# Наилучшие гиперпараметры
best_params = clf.best_params_

# Обучение модели с наилучшими параметрами
best_clf = svm.SVC(**best_params, probability=True)
best_clf.fit(X_train, y_train)

SVC(C=1, kernel='linear', probability=True)

In [365]:
from joblib import dump
dump(best_clf, 'final_model.joblib', compress=9)

['final_model.joblib']

In [362]:
# Оценка точности модели на валидационном наборе
y_val_pred = best_clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Точность модели на валидационном наборе данных: {:.2f}".format(val_accuracy))

Точность модели на валидационном наборе данных: 0.92


In [363]:
# Оценка точности модели на тестовом наборе данных
y_test_pred = best_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Точность модели на тестовом наборе данных: {:.2f}".format(test_accuracy))
print('Precision:', precision_score(y_test, y_test_pred, average='weighted'))
print('Recall:', recall_score(y_test, y_test_pred, average='weighted'))
print('F1:', f1_score(y_test, y_test_pred, average='weighted'))

Точность модели на тестовом наборе данных: 0.94
Precision: 0.9420940144121723
Recall: 0.9417040358744395
F1: 0.9416241564427388


In [364]:
# testing of the single image
features = []
derect = 'D:/Практика/test_txt'
for file_name in os.listdir(derect):
    img_path = os.path.join(derect, file_name)
    img_array = imread(img_path, as_gray=False)
    if len(img_array.shape) != 3 or img_array.shape[2] != 3:
      img_array =  cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    size = (250, 250)
    resized_img = cv2.resize(img_array, size)
    #bright_img = cv2.add(resized_img, 50)
    img_tensor = torch.tensor(resized_img).permute(2, 0, 1)[None, ...] / 255
    img_feats = fid_metric.features(img_tensor).reshape(-1)
    features.append(img_feats)

print(os.listdir(derect))
print(best_clf.predict_proba(features))

['7b32934e-34c9-4731-a8d3-f207bd9d0ab4.jpg', 'a5dfba53-b7df-4151-b047-2d9c1f8faf7b.jpg', 'a906beff-2ad9-45a4-afdd-a6eac4f11654.jpg', 'b903001d-d184-46ff-bdcb-6b48f1ceb244.jpg', 'e5996ae0-e2df-4705-840e-428aee1136fc.jpg', 'Больше нечего 💫 холст_акрил под заказ.jpg', 'Дом, где пахнет шарлоткой_ О теплых встречах, женской дружбе и мечтах, которые вредно откладывать.jpg', '💫💗.jpg']
[[6.95168813e-02 4.66294025e-02 8.83853716e-01]
 [9.26940363e-02 1.78515016e-02 8.89454462e-01]
 [2.04959929e-03 8.87162764e-03 9.89078773e-01]
 [1.46808521e-02 5.99696740e-03 9.79322181e-01]
 [3.78622889e-01 8.87109415e-03 6.12506017e-01]
 [1.55578852e-06 1.85872627e-06 9.99996585e-01]
 [1.91703170e-03 3.06222941e-01 6.91860028e-01]
 [1.27098230e-02 2.16383978e-03 9.85126337e-01]]


In [370]:
np.argmax(best_clf.predict_proba(features[0].reshape(1,-1)) )

2

In [318]:
best_clf.predict(fid_metric.features(img_tensor).reshape(-1).reshape(1,-1)).astype(int)

array([0])